In [1]:
import os
if not os.path.basename(os.getcwd()) in ["ea-nas", "EA-architecture-search"]:
    os.chdir("../")


import sys
import pandas as pd
import LAB.common as fn
import LAB.report_functions as rfn
from src.configuration import Configuration

import plotly
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)

# INTERPRETATION OF EA-NAS RESULTS
This notebook is meant to interpret the results of simulations ran using EA-NAS.
* Analyzes multiple individs
* Groups individs by their versions

Look at the common.py file for more functions to use for analysis

## Load in simulation results: 

Set the correct paths to: 
* Configuraiton file with settings for the simulation
* Path to results save directory

In [2]:
# Configuration file:
config_file = "./LAB/notebook.json"

In [3]:
# Load configuration used:
config = Configuration.from_json(config_file)

# Loads all modules from a given run:
all_modules_grouped = fn.load_all_modules_from_run(f"./results/{config.results_name}")

# Reporting on what modules are loaded: 
print(f"Loaded {sum([len(val) for val in all_modules_grouped.values()])} modules")

Loaded 110 modules


In [4]:
reports = {}
for name, modules in all_modules_grouped.items():
    reports[name] = {
        m.version: fn.progress_report(m.report, m.ID) 
        for m in modules if m.report
    }
print(f"Loaded {sum([len(val) for val in reports.values()])} reports")

Loaded 76 reports


In [5]:
# Modules loaded: 
for key, val in all_modules_grouped.items():
    print(f"\t{key} with versions [v{', v'.join([str(m.version) for m in val])}]")

	Tone with versions [v0]
	Leonora with versions [v0]
	Mahad with versions [v0, v2, v4]
	Ine with versions [v0]
	Pedro with versions [v0, v2]
	Bergit with versions [v0]
	Asma with versions [v0]
	Elisa with versions [v6, v8, v0, v2, v4]
	Rolandas with versions [v0]
	Maximilian with versions [v0]
	Vera with versions [v0, v2]
	Toralf with versions [v0]
	Berit with versions [v0, v2, v4]
	Henny with versions [v0]
	Tiril with versions [v0]
	Arild with versions [v0]
	Agnar with versions [v0]
	Galina with versions [v6, v8, v0, v2, v4]
	Thor with versions [v0]
	Borgar with versions [v0, v2]
	Rebekka with versions [v0, v2, v4]
	Iris with versions [v6, v8, v0, v2, v4]
	Egil with versions [v0]
	Torgeir with versions [v0]
	Judit with versions [v0]
	Preben with versions [v0]
	Jo with versions [v0, v2]
	Sophia with versions [v0]
	Ricardo with versions [v0, v2]
	Robert with versions [v0]
	Katrine with versions [v0]
	Ann-karin with versions [v0]
	Anne-mari with versions [v0]
	Mette with versions [v0, v2

## Interpreting results per training session: 
Looking at the average gains/losses of performance per class over multiple training sessions below. Gains are positive numbers while losses are negative... 

In [6]:
# Changes over training sessions:
diff_dicts = [
    rfn.score_changes_per_training_session(df) 
    for df in rfn.find_multi_session_reports(reports)
]
pd.concat(diff_dicts).mean()

0, Airplane     -0.087494
1, Automobile   -0.089008
2, Bird         -0.061009
3, Cat          -0.031157
4, Deer         -0.053435
5, Dog          -0.066961
6, Frog         -0.076448
7, Horse        -0.076904
8, Ship         -0.068366
9, Truck        -0.074044
micro avg       -0.062773
macro avg       -0.068483
weighted avg    -0.068483
dtype: float64

### Plot of change per training session
Splitting reports by training session number:

In [7]:
multi_session_reports = list(rfn.find_multi_session_reports(reports))

## Looking into training history for a single individ:
* Subject of interst is the specimin 'Galina'
* Contains 5 versions

In [8]:
versions = all_modules_grouped['Anne-sofie']
versions.sort(key=lambda x: x.version)
print([x.ID for x in versions])
v0 = versions[0] # Selecting Galina v0

['Anne-sofie v0', 'Anne-sofie v2', 'Anne-sofie v4']


In [9]:
def generate_training_data_plot(module, validation=True, training=False):
    labels = tuple(range(1, len(module.validation_fitness)+1))
    traces = []
    if validation:
        data = tuple(module.validation_fitness)
        traces += [go.Scatter(x=labels, y=data, mode="lines+markers", name=f"{module.ID} Validaiton set")] 
    if training: 
        data = tuple(module.fitness)
        traces += [go.Scatter(x=labels, y=data, mode="lines+markers", name=f"{module.ID} Training set")] 
    return traces

In [10]:
traces = []
for v in versions: traces += generate_training_data_plot(v)
plotly.offline.iplot({"data": traces })

### Validation accuary plot for transerring leaning

In [22]:
def generate_training_plot_for_transfer_learning(modules):
    traces = []
    prev_end_training = 0
    for i, module in enumerate(tree):
        labels = tuple(range(prev_end_training, len(module.validation_fitness) + prev_end_training))
        trn = module.fitness
        val = module.validation_fitness
        traces += [go.Scatter(x=labels, y=trn, mode="lines+markers", name=f"Training, {module.name} v{i}")]
        traces += [go.Scatter(x=labels, y=val, mode="lines+markers", name=f"Validation, {module.name} v{i}")]
        prev_end_training = len(tree[0].validation_fitness) -1
    return traces


def find_longest_predecessor_range(modules):
    def dive(module):
        res = [module]
        if module.predecessor:
            return dive(module.predecessor) + res
        return res
    _max = 0
    longest_tree = []
    for module in modules:
        tree = dive(module)
        if len(tree) > _max:
            _max = len(tree)
            longest_tree = tree
    
    return longest_tree
    
    
tree = find_longest_predecessor_range(versions)
baseline = go.Scatter(y=tuple([0.82]*31), x=tuple(range(0, 31)), name="Baseline")
plotly.offline.iplot({
    "data": generate_training_plot_for_transfer_learning(tree) + [baseline], 
    "layout": go.Layout()
})

In [12]:
fn.create_images(tree, config)

TypeError: _save_kears_model_images() takes 1 positional argument but 3 were given

In [ ]:
## classified = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
for epoch_generated, report in v0.report.items():
    print(report[classified[3]]['precision'])

In [11]:
plotly.offline.iplot({
     traces,
    "layout": go.Layout(barmode='group', yaxis=dict())
})